<a href="https://colab.research.google.com/github/sunyingjian/study-tensorflow/blob/master/%E7%8C%AB%E7%8B%97%E6%95%B0%E6%8D%AE%E9%9B%86%E8%87%AA%E5%AE%9A%E4%B9%89%E8%AE%AD%E7%BB%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
%tensorflow_version 2.x
import tensorflow as tf
import os
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/My Drive/dataset"
os.chdir(path)
os.listdir(path)
import numpy as np


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import glob#提取图像的路径

In [0]:
train_image_path=glob.glob('/content/drive/My Drive/dataset/dc_2000/train/*//*.jpg')

In [31]:
len(train_image_path)

2000

In [32]:
train_image_path[:5]

['/content/drive/My Drive/dataset/dc_2000/train/cat/cat.0.jpg',
 '/content/drive/My Drive/dataset/dc_2000/train/cat/cat.1.jpg',
 '/content/drive/My Drive/dataset/dc_2000/train/cat/cat.10.jpg',
 '/content/drive/My Drive/dataset/dc_2000/train/cat/cat.100.jpg',
 '/content/drive/My Drive/dataset/dc_2000/train/cat/cat.101.jpg']

In [33]:
train_image_path[-5:]

['/content/drive/My Drive/dataset/dc_2000/train/dog/dog.997.jpg',
 '/content/drive/My Drive/dataset/dc_2000/train/dog/dog.998.jpg',
 '/content/drive/My Drive/dataset/dc_2000/train/dog/dog.999.jpg',
 '/content/drive/My Drive/dataset/dc_2000/train/dog/dog.992.jpg',
 '/content/drive/My Drive/dataset/dc_2000/train/dog/dog.993.jpg']

##进行路径拆分，到他到底是猫还是狗

In [34]:
p='/content/drive/My Drive/dataset/dc_2000/train/dog/dog.993.jpg'
p.split('/')[7]

'dog'

In [0]:
#我们这里就可以实现把列表的dog和cat变成0和1这样就实现了trainlabel
#这样cat就会变成1 因为是转换成了int型
train_image_label=[int(p.split('/')[7]=='cat') for p in train_image_path]

In [36]:
train_image_label[-5:]

[0, 0, 0, 0, 0]

##定义函数读取路径

In [0]:
def load_preprosess_image(path,label):
  image=tf.io.read_file(path)#读取这个路径
  image=tf.image.decode_jpeg(image,channels=3)#解码数据
  #把图像划分到同样大小
  image=tf.image.resize(image,[256,256])
  #改变数据类型把uint8变成float32
  image=tf.cast(image,tf.float32)
  #归一化
  image=image/255
  #处理标签,把连着的label断开
  #[1,2,3]→[1][2][3]这个样子
  label=tf.reshape(label,[1])
  return image,label

##创建数据集

In [0]:
#创建数据集
train_image_ds=tf.data.Dataset.from_tensor_slices((train_image_path,train_image_label))
AUTOTUNE=tf.data.experimental.AUTOTUNE
#实现针对于cpu智能的并行的创建数据集
train_image_ds=train_image_ds.map(load_preprosess_image,num_parallel_calls=AUTOTUNE)

In [39]:
train_image_ds

<ParallelMapDataset shapes: ((256, 256, 3), (1,)), types: (tf.float32, tf.int32)>

In [0]:
#设置batchsize
BATCH_SIZE=128#根据性能测试
train_count=len(train_image_path)
train_image_ds=train_image_ds.shuffle(train_count).batch(BATCH_SIZE)
#为了加速训练过程
train_image_ds=train_image_ds.prefetch(AUTOTUNE)

In [0]:
#这里的化他是一个eage模式的 他是可以迭代的，我们尝试着从中取出一个看一下
image,labels=next(iter(train_image_ds))

In [42]:
image.shape

TensorShape([128, 256, 256, 3])

In [43]:
labels.shape

TensorShape([128, 1])

##创建模型

In [0]:
model=tf.keras.Sequential([
    tf.keras.layers.Conv2D(64,(3,3),input_shape=(256,256,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(256,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(512,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(1024,(3,3),activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dense(1)
])

In [45]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 254, 254, 64)      1792      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 127, 127, 64)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 125, 125, 128)     73856     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 62, 62, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 60, 60, 256)       295168    
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 30, 30, 256)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 28, 28, 512)      

In [0]:
pred=model(image)

In [47]:
tf.cast(pred>0,tf.int32)

<tf.Tensor: shape=(128, 1), dtype=int32, numpy=
array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

In [48]:
np.array(p[0].numpy() for p in tf.cast(pred>0,tf.int32))

array(<generator object <genexpr> at 0x7fe305f040f8>, dtype=object)

In [49]:
labels

<tf.Tensor: shape=(128, 1), dtype=int32, numpy=
array([[1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
    

In [50]:
np.array(p[0].numpy() for l in labels)

array(<generator object <genexpr> at 0x7fe305f04ba0>, dtype=object)

##自定义损失函数

In [0]:
ls=tf.keras.losses.BinaryCrossentropy()#大写的是返回的可调用的函数

In [52]:
ls([0.,0.,1.,1.],[1.,1.,1.,1.])

<tf.Tensor: shape=(), dtype=float32, numpy=7.6666193>

In [53]:
ls=tf.keras.losses.binary_crossentropy([0.,0.,1.,1,],[1.,1.,1.,1.])
print(ls)

tf.Tensor(7.6666193, shape=(), dtype=float32)


#计算均值与正确率

In [0]:
epoch_loss_avg=tg.keras.metrics.Mean('trian_loss')
train_accuracy=tf.keras.metrics.Accuracy()

##自定义优化器

In [0]:
optimizer=tf.keras.optimizers.Adam()
#定义一步优化函数
def train_step(model,images,labels):
  with tf.GradientTape() as t:#创建一个上下文管理器
    pred=model(images)#pred就是我们要预测的值
    loss_step=tf.keras.losses.BinaryCrossentropy(from_logits=True)(label,pred)
  grads=t.gradient(loss_step,model.trainable_variables)#这里是在计算梯度
  optimizer.apply_gradients(zip(grads,model.trainable_variables))